# АВ testing Cookie cats

## Part 1. Data exploration

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/mobile-games-ab-testing-cookie-cats/cookie_cats.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df_info = df.groupby('version')['userid'].nunique().to_frame()
df_info

## Part 2. Setting the hypothesis
Let's look at the received data to form a null hypothesis.

In [ ]:
df_ab = df.groupby('version').agg({'retention_1':'mean', 'retention_7': 'mean'})
df_ab

At first glance, we can see that the control group has the best retention. Based on this, form a null and alternative hypothesis.
>H0:  A > B

>H1:  A <= B

## Part 3. Bootstrap 

In [ ]:
diff = df_ab.iloc[0] - df_ab.iloc[1]
diff.to_frame()

In [ ]:
bt_1d = []
bt_7d = []
for i in range(1000):
    sel_1d = df.sample(frac = 1, replace=True).groupby('version')['retention_1'].mean().to_frame()
    sel_7d = df.sample(frac = 1, replace=True).groupby('version')['retention_7'].mean().to_frame()
    bt_1d.append(sel_1d.iloc[0] - sel_1d.iloc[1])
    bt_7d.append(sel_7d.iloc[0] - sel_7d.iloc[1])
                 
df_1d = pd.DataFrame(bt_1d)
df_7d = pd.DataFrame(bt_7d)

In [ ]:
fig, ax = plt.subplots(1,2)
p1 = sns.histplot(df_1d, ax=ax[0])
p2 = sns.histplot(df_7d, ax=ax[1])
ax[0].axvline(diff[0], color='r', linestyle='--')
ax[1].axvline(diff[1], color='r', linestyle='--')
fig.show()

In [ ]:
(df_1d['retention_1'] >= diff[0]).mean()

In [ ]:
(df_7d['retention_7'] >= diff[1]).mean()

## Conclusion 

Thus, by setting the significance level to 5%, we can assert that the null hypothesis is correct. The control group has the best retention for both the 1st day and the 7th day, so innovation should not be introduced into this product.